### The Project

The Project
The goals / steps of this project are the following:

-Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
-Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
-Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
-Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
-Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
-Estimate a bounding box for vehicles detected.

In [ ]:
### Gather and combine all data for vehcile and non - vehicles

In [ ]:
import os
import glob

base = './datasets/vehicles/'
folders = os.listdir(base)

cars = []
for folder in folders:
    cars.extend(glob.glob(base+folder+'/*'))

base = './datasets/non-vehicles/'
folders = os.listdir(base)

notcars = []
for folder in folders:
    notcars.extend(glob.glob(base+folder+'/*'))
    
print("Number of Vehicle Images found: {}".format(len(cars)))
print("Number of Non-Vehicle Images found: {}".format(len(notcars)))